# Préparation

In [ ]:
# Clonage du repo GitHub et installation des prérequis
!git clone https://github.com/ultralytics/yolov5
!pip install -qr yolov5/requirements.txt

In [ ]:
%cd /content/yolov5

In [ ]:
import torch
print('On utilise PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'Pas de GPU disponible'))

# Téléchargement de la BDD sur Roboflow

In [ ]:
# Mirror
!curl -L "https://app.roboflow.com/ds/ASw88RQ89x?key=s5JKgpFV9x" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
# Camera
!curl -L "https://app.roboflow.com/ds/ZsxWeFDiVU?key=YGUAWy330B" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
# Camera 2
!curl -L "https://app.roboflow.com/ds/sQiLyUrJaz?key=EMIcaPrwGd" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

# Train

In [5]:
# Modification de data.yaml pour fonctionner sur colab
import os
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [6]:
%%writetemplate /content/yolov5/data.yaml

train: ./train/images
val: ./valid/images

nc: 1
names: ['person']

In [ ]:
# Installation de WandB pour le suivi de l'entrainement
!pip install wandb

In [ ]:
# Entrainement du modèle
# On rentre des images en 640x640
# Batch : 64 pour S ; 32 pour M ; 16 pour L ; 8 pour X
# Epochs : 100 - 300
# On choisit des poids déjà entrainés sur coco
# Les nouveaux poids seront stockés ici /content/yolov5/runs/exp/weights/best.pt
%cd /content/yolov5/
!python train.py --img 640 --batch 8 --epochs 10 --data data.yaml --weights yolov5l6.pt

# Test

In [ ]:
%cd /content/yolov5
!gdown 1-P9wmrDBlLKNKVLfSsKqOc0a_Wu3FtAB
!python val.py --weights /content/yolov5/YOLOv5x62.pt --data data.yaml --img 640 --conf-thres 0.3

In [ ]:
# On teste la précision des poids
%cd /content/yolov5
!python val.py --weights /content/yolov5/runs/train/exp/weights/best.pt --data data.yaml --img 640

In [ ]:
# On détecte sur une vidéo pour tester les poids
!python detect.py --source /content/yolov5/test/images --weights runs/train/exp/weights/best.pt